# 1 线性层公式

## 1.1 线性性层的前向传播
$$
\boldsymbol{Y}=\boldsymbol{XW}+\left( \boldsymbol{1}_{p\times 1} \right) \boldsymbol{b} \tag{1.1}
$$

其中：
$X$是输入 : (batch_size, input_size)
$W$是权重 : (input_size, output_size)
$b$是偏置 : (1, output_size)
$Y$是输出 : (batch_size, output_size)
$p$是batch_size, $(\boldsymbol{1}_{p\times 1})$表示$b$会在batch_size上进行广播

## 1.2 线性性层的反向传播
### 1.2.1 对权重矩阵$W$的梯度
$$ 
\nabla _{\boldsymbol{W}}L=\left( \boldsymbol{X}^{\top} \right) \left( \nabla _{\boldsymbol{Y}}L \right) \tag{1.2}
$$

其中：
$X^T$是输入的转置 : (input_size, batch_size)
$\nabla _{\boldsymbol{Y}}L$是损失函数对本层输出的梯度 : (batch_size, output_size)
$\nabla _{\boldsymbol{W}}L$是损失函数对本层权重的梯度 : (input_size, output_size)

### 1.2.2 对偏置$b$的梯度
$$
\nabla _{\boldsymbol{b}}L=\left( \boldsymbol{1}_{1\times p} \right) \left( \nabla _{\boldsymbol{Y}}L \right) \tag{1.3}
$$

其中：
$\nabla _{\boldsymbol{Y}}L$是损失函数对本层输出的梯度 : (batch_size, output_size)
$p$是batch_size, $(\boldsymbol{1}_{1\times p})$表示$\nabla _{\boldsymbol{b}}L$会在batch_size上进行反向广播，即求和
$\nabla _{\boldsymbol{b}}L$是损失函数对本层偏置的梯度 : (1, output_size)

### 1.2.3 对输入$X$的梯度
$$
\nabla _{\boldsymbol{X}}L=\left( \nabla _{\boldsymbol{Y}}L \right)\left( \boldsymbol{W}^{\top} \right) \tag{1.4}
$$

其中：
$\nabla _{\boldsymbol{Y}}L$是损失函数对本层输出的梯度 : (batch_size, output_size)
$W^T$是权重的转置 : (output_size, input_size)
$\nabla _{\boldsymbol{X}}L$是损失函数对本层输入的梯度 : (batch_size, input_size)

## 1.3 线性性层的代码实现

In [26]:
import numpy as np


class LinearLayer:
    def __init__(self, input_size, output_size):
        # 初始化权重矩阵和偏置向量
        self.weights = np.random.normal(loc=0.0, scale=0.1, size=(input_size, output_size))
        self.bias = np.zeros((1, output_size))

        # 存储输入输出用于反向传播
        self.x = None

        # 存储梯度用于反向传播
        self.d_weights = None
        self.d_bias = None

    def forward(self, x):
        # 备份输入x，反向传播时会用到
        self.x = x
        # 前向传播计算输出，对应公式(1.1)
        # bias会自动broadcasting到(batch_size, output_size)维
        output = np.matmul(self.x, self.weights) + self.bias
        return output

    def backward(self, d_output):
        # 反向传播计算权重的梯度，对应公式(1.2)
        self.d_weights = np.matmul(self.x.T, d_output)
        # 反向传播计算偏置的梯度，对应公式(1.3)
        self.d_bias = np.sum(d_output, axis=0)
        # 反向传播计算输入的梯度，对应公式(1.4)
        d_input = np.matmul(d_output, self.weights.T)
        return d_input

    def update(self, learning_rate):
        # 使用梯度下降法更新权重和偏置
        self.weights -= learning_rate * self.d_weights
        self.bias -= learning_rate * self.d_bias

# 2 ReLU激活函数层公式
# 2.1 ReLU激活函数层的前向传播
$$
\boldsymbol{Y}=\max \left( \boldsymbol{X}, 0 \right) \tag{2.1}
$$

其中：
$X$是输入 : (batch_size, input_size)
$Y$是输出 : (batch_size, input_size)
一句话描述：将$X$中小于0的值置为0，大于等于0的值保持不变

## 2.2 ReLU激活函数层的反向传播
### 2.2.1 对输入$X$的梯度
$$
\nabla _{\boldsymbol{X}}L=\begin{cases} 	0,&x<0\\ 	\nabla _{\boldsymbol{Y}}L,&x\geqslant 0\\ \end{cases} \tag{2.2}
$$

其中：
$\nabla _{\boldsymbol{Y}}L$是损失函数对本层输出的梯度 : (batch_size, input_size)
$\nabla _{\boldsymbol{X}}L$是损失函数对本层输入的梯度 : (batch_size, input_size)
一句话描述：$X$中小于0的值对应的梯度为0，大于等于0的值对应的梯度为$\nabla _{\boldsymbol{Y}}L$

## 2.3 ReLU激活函数层的代码实现

In [27]:
class ReLULayer(object):
    def __init__(self):
        self.x = None

    def forward(self, x):
        # 备份输入x，反向传播时会用到
        self.x = x
        # 前向传播计算，对应公式(2.1)
        output = np.maximum(0, x)
        return output

    def backward(self, d_output):
        # 反向传播的计算，对应公式(2.2)
        d_input = d_output
        d_input[self.x < 0] = 0
        return d_input

# 3 Softmax函数公式
## 3.1 Softmax函数的前向计算
$$
\boldsymbol{\hat{Y}}_{i,j}  =\frac{\exp \left[ \boldsymbol{X}_{i,j} \right]}{\displaystyle \sum_j{\exp \left[ \boldsymbol{X}_{i,j} \right]}} \tag{3.1}
$$

其中：
$X$是已经通过线性层从(batch_size, hidden_size)映射到(batch_size, num_class)的输入
$\hat{Y}$是输出 : (batch_size, num_class), $\hat{Y}_{i,j}$表示第$i$个样本属于第$j$类的概率

考虑到${X}_{i,j}$较大时，$\exp \left[ \boldsymbol{X}_{i,j} \right]$可能会溢出，所以我们可以对公式(3.1)稍作变形：
$$
\boldsymbol{\hat{Y}}_{i,j}=\frac{\displaystyle \exp \left[ \boldsymbol{X}_{i,j}-\max_n \boldsymbol{X}_{i,n} \right]}{\displaystyle \sum_j{\exp \left[ \boldsymbol{X}_{i,j}-\max_n \boldsymbol{X}_{i,n} \right]}} \tag{3.2}
$$
对于每一个样本，使其特征向量中每一项都减去最大项$\max_n \boldsymbol{X}_{i,n}$不会改变结果，但是可以避免溢出

## 3.2 Softmax函数的反向传播
### 3.2.1 对输入$X$的梯度
$$
\frac{\partial \boldsymbol{\hat{y}}_{i,k}}{\partial \boldsymbol{x}_{i,j}}=\frac{\partial}{\partial \boldsymbol{x}_{i,j}}\frac{\exp \left[ \boldsymbol{X}_{i,k} \right]}{\sum_k{\exp \left[ \boldsymbol{X}_{i,k} \right]}}=\begin{cases} 	-\boldsymbol{\hat{y}}_{i,j}\boldsymbol{\hat{y}}_{i,k},&k\ne j\\ 	-\boldsymbol{\hat{y}}_{i,j}\boldsymbol{\hat{y}}_{i,k}+\boldsymbol{\hat{y}}_{i,j},&k=j\\ \end{cases} \tag{3.3}
$$
其中：
$\boldsymbol{\hat{y}}_{i,k}$表示样本$i$属于类别$k$的概率，其在对特征向量$\boldsymbol{x}_{i}$中的每一项求偏导数时，根据softmax函数的公式与函数除法的求导法则，可以得知，在${x}_{i,j}$项只存在于分母时和${x}_{i,j}$项同时存在于分子和分母时，对应的偏导数不同

## 3.3 Softmax函数的代码实现

In [28]:
def softmax(input):
    # 通过softmax函数计算概率
    # 减去输入的最大值，防止指数爆炸
    # 对应公式(3.2)
    input_max = np.max(input, axis=1, keepdims=True)
    input_exp = np.exp(input - input_max)
    # 计算概率
    prob = input_exp / np.sum(input_exp, axis=1, keepdims=True)
    return prob
# 反向传播的代码，与后面的交叉熵损失函数结合实现

## 4 交叉熵损失函数公式
## 4.1 交叉熵损失函数的前向计算
$$
L=-\frac{1}{p}\sum_i{\boldsymbol{y}_i\ln \boldsymbol{\hat{y}}_i} \tag{4.1}
$$

其中：
$p$是batch_size
$\boldsymbol{y}_i$是一个$num\_class$维的one-hot向量, 其中样本$i$的真实标签对应的位置为1，其他位置为0
$\boldsymbol{\hat{y}}_i$是softmax层的输出，表示样本$i$属于各个类别的概率
$\boldsymbol{y}_i\ln \boldsymbol{\hat{y}}_i$相当于只保留了样本$i$属于真实标签类别的概率

考虑到${{\hat{y}}_i}$趋近于0时，$\ln{{\hat{y}}_i}$可能会发生溢出，在代码实现时一般会结合防溢出softmax函数（公式(3.2)）一起使用：
$$
\begin{aligned}
\log{(\hat y_j)} & = \log\left( \frac{\exp(o_j - \max(o_k))}{\sum_k \exp(o_k - \max(o_k))}\right) \\
& = \log{(\exp(o_j - \max(o_k)))}-\log{\left( \sum_k \exp(o_k - \max(o_k)) \right)} \\
& = o_j - \max(o_k) -\log{\left( \sum_k \exp(o_k - \max(o_k)) \right)}
\end{aligned} \tag{4.2}
$$

其中：
$\hat{y}$是一个$num\_class$维的向量，$\hat{y}_j$表示样本属于类别$j$的概率
$\log$实际上是以$e$为底的对数函数，即$\ln$
$o_j$是softmax层的输入，即未经过softmax函数的model的输出
$\max(o_k)$是维数为$k$的样本特征向量$o$中的最大项

## 4.2 交叉熵损失函数的反向传播
### 4.2.1 对输入$X$的梯度

$$
\begin{aligned} 	
\left( \nabla _{\boldsymbol{X}}L \right) _{i,j}&=\frac{\partial L}{\partial x_{i,j}}=\frac{1}{p}\left( \boldsymbol{\hat{y}}_{i,j}-\boldsymbol{y}_{i,j} \right)\\
\end{aligned}\tag{4.3}
$$

详细推导过程如下：
$$
\frac{\partial L}{\partial x_{i,j}}=-\frac{1}{p}\sum_k{\frac{\boldsymbol{y}_{i,k}}{\boldsymbol{\hat{y}}_{i,k}}\frac{\partial \boldsymbol{\hat{y}}_{i,k}}{\partial x_{i,j}}}
$$
根据softmax的求导公式(3.3)，拆分$\frac{\partial \boldsymbol{\hat{y}}_{i,k}}{\partial x_{i,j}}$为两种情况：
$$
=-\frac{1}{p}(\sum_{k\neq j}{\frac{\boldsymbol{y}_{i,k}}{\boldsymbol{\hat{y}}_{i,k}} (-\boldsymbol{\hat{y}}_{i,j}\boldsymbol{\hat{y}}_{i,k})+{\frac{\boldsymbol{y}_{i,j}}{\boldsymbol{\hat{y}}_{i,j}}(-\boldsymbol{\hat{y}}_{i,j}\boldsymbol{\hat{y}}_{i,j}+\boldsymbol{\hat{y}}_{i,j}))
$$
补齐缺项求和
$$
=\frac{1}{p}(\sum_k{\frac{\boldsymbol{y}_{i,k}}{\boldsymbol{\hat{y}}_{i,k}}\boldsymbol{\hat{y}}_{i,j}\boldsymbol{\hat{y}}_{i,k}}-{\frac{\boldsymbol{y}_{i,j}}{\boldsymbol{\hat{y}}_{i,j}}\boldsymbol{\hat{y}}_{i,j})
$$
化简分母
$$
=\frac{1}{p}((\sum_k{{\boldsymbol{y}_{i,k}})\boldsymbol{\hat{y}}_{i,j}-\boldsymbol{y}_{i,j})
$$
$\sum_k{{\boldsymbol{y}_{i,k}}=1$, 所以
$$
=\frac{1}{p}\left( \boldsymbol{\hat{y}}_{i,j}-\boldsymbol{y}_{i,j} \right)
$$

## 4.3 交叉熵损失函数的代码实现

In [29]:
class CrossEntropyLossLayer:
    def __init__(self):
        self.prob = None
        self.label_onehot = None

    def forward(self, output, label):
        # 备份概率值，反向传播时会用到
        self.prob = softmax(output)
        # 将标签转换为one-hot编码并备份
        batch_size = self.prob.shape[0]
        self.label_onehot = np.zeros_like(self.prob)
        self.label_onehot[np.arange(batch_size), label] = 1.0
        # 计算交叉熵损失，对应公式(4.1)
        # loss = -np.sum(np.log(self.prob) * self.label_onehot) / batch_size
        # 为了防止溢出，使用下面的计算方式，对应公式(4.2)
        output_max = np.max(output, axis=1, keepdims=True)
        log_prob = output - output_max - np.log(np.sum(np.exp(output - output_max), axis=1, keepdims=True))
        loss = -np.sum(log_prob * self.label_onehot) / batch_size
        return loss

    def backward(self):
        # 反向传播计算输入的梯度，对应公式(4.3)
        # 虽然我们在前向计算时使用了优化过的公式，但这些修改不影响梯度的计算
        batch_size = self.prob.shape[0]
        d_input = (self.prob - self.label_onehot) / batch_size
        return d_input

# 参考资料：
https://zhuanlan.zhihu.com/p/380036598  
https://zh.d2l.ai/chapter_multilayer-perceptrons/backprop.html  
https://zh.d2l.ai/chapter_linear-networks/softmax-regression-concise.html  
https://www.cnblogs.com/gczr/p/16345902.html  
https://blog.csdn.net/chaipp0607/article/details/101946040  